### Einleitung

In den Straßen New York Citys herrscht ein hohes Verkehrsaufkommen, wodurch Verkehrsunfälle an der Tagesordnung sind. Durch [NYC Open Data](https://opendata.cityofnewyork.us/) werden die erfassten Verkehrsunfälle für die Allgemeinheit zugänglich gemacht.

Thema dieses Berichtes und Thema unseres Projektes ist es, die öffentlich zugänglichen Unfalldaten zu nutzen, um darauf die im Abschnitt `Analyseziele` beschriebenen Analysen durchzuführen und die Ergebnisse zu visualisieren.

## Ausführungsplan

Bevor der eigentliche Bericht und der entsprechende Code genauer behandelt werden, wird hier erläutert, was installiert und beachtet werden muss, um einen möglichst reibungslosen Ablauf zu garantieren.

### Installation der benötigten Docker-Container

Damit keine weiteren Komponenten manuell installiert werden müssen und keine Änderungen am ausführenden System entstehen, werden alle Komponenten durch [Docker](https://docs.docker.com/get-docker/) installiert und ausgeführt. Somit wird Docker benötigt und muss installiert werden.

Die benötigten Docker-Container sind in der Docker-Compose-Konfiguration (`docker-compose.yml`) definiert.
Diese können mit `docker-compose up -d` heruntergeladen und gestartet werden. 
Mit `docker-compose down` können die gestarteten Container gestoppt werden. Das Terminal sollte nicht geschlossen werden, bevor alle Container durch `docker-compose down` gestoppt vollständig gestoppt wurden.

Durch Nutzung einer angepassten `docker-compose.yml` konnten Fehler, die mit der Verwendung einer vorgefertigten `docker-compose.yml` entstanden sind, ausgeräumt werden.

### Starten des Programmcodes

Nachdem alle Container über `docker-compose up -d` gestartet wurden, kann mit der Ausführung des Jupyter Notebooks begonnen werden. Hierbei kann unter `Kernel` die Option `Restart & Run All` genutzt werden, um das komplette Notebook auszuführen. Sollte der Code nicht auf einem Windows-System ausgeführt werden, so befindet sich im Kapitel `Starten des Join-Prozesses (außer auf Windows)` eine Art Breakpoint. Dieser Breakpoint dient (wie unten beschrieben) dazu, dass der Join-Prozess nicht ausgeführt wird, bevor alle notwendigen Daten geladen sind. Zur Ausführung des Codes unter dem Breakpoint sind die Anweisungen am Anfang des entsprechenden Kapitels zu befolgen.

## Aufbau der Data Pipeline

Für das Projekt soll eine Data-Pipeline aufgebaut werden, welche die Daten in die Datenbank lädt. 
Von dort aus sollen die Daten von einem Python-Script zur Analyse angefragt werden.

Beteiligt an der Data-Pipeline ist eine Kafka-Instanz, wobei ein Python-Script einen Producer dafür darstellt und ein weiteres Python-Script einen Consumer. Das Producer-Script lädt die Daten zeilenweise aus der Datenquelle, einer CSV-Datei, in Kafka ein, während der Consumer die Daten aus Kafka ausliest und in die MongoDB schreibt. Das Analyse-Script bezieht die Daten wiederum aus der MongoDB.

## Datenquellen

Als Datenquellen dienen die vom NYC Open Data Project bereitgestellten Unfalldaten des NYPD. Die detaillierten Unfalldaten befinden sich in drei unterschiedlichen Datenquellen die unter Anderem als CSV-Datei verfügbar sind: [Motor Vehicle Collisions](https://data.cityofnewyork.us/browse?Data-Collection_Data-Collection=Motor+Vehicle+Collisions&q=crashes).

Enthalten in den Daten sind die am Unfall beteiligten Verkehrsteilnehmer, die Verletzten und Toten, sowie die genaue Position des Unfalls. Darüberhinaus sind noch weitere Daten in den Quellen enthalten, auf die teilweise im Analyseabschnitt genauer eingegangen wird.

## Analyseziele

Nach der Verarbeitung und Aufbereitung der Daten soll aus der Analyse hervorgehen, welche saisonalen und lokalen Zusammenhänge aus den Unfalldaten gelesen werden können. Des Weiteren soll das Ergebnis eine Aussage darüber geben, ob sich bestimmte Unfallhotspots innerhalb von New York lokalisieren lassen und ob diese Hotspots sich mit der Zeit ändern. Während der Ansicht der Unfallhotspots soll es auch möglich sein, zu erkennen, an welchem Hotspot sich besonders viele schwerwiegende Unfälle ereignen (Unfälle mit vielen verletzten oder getöteten Verkehrsteilnehmern). Zuletzt soll aus der Analyse hervorgehen, welche Verkehrsteilnehmer besonders oft getötet und verletzt werden. Hierzu zählen sowohl die Füßgänger, die Fahrradfahrer als auch die Fahrer von motorisierten Fahrzeugen.

Die Visualisierung der Daten erfolgt über verschiedene Diagramme sowie eine Heatmap, welche mehrere Filtermöglichkeiten bereitstellt.

## Installation der Python Packages

Damit nicht alles vollständig von uns entwickelt werden muss, wurden verschiedene Bibliotheken verwendet. Auf diese Weise kann sich die Umsetzung auf das Wesentliche beziehen.

[Apache Kafka](https://kafka.apache.org/) soll dazu genutzt werden, mit Hilfe eines Producers und eines Consumers die Datensätze zeilenweise einzulesen. Dies wird durch die [kafka-python](https://github.com/dpkp/kafka-python) Bibliothek stark vereinfacht.

Während der Kafka Consumer die Datensätze registiert und ausliest müssen diese ein die MongoDB eingetragen werden. Um dieses Verfahren zu vereinfachen, wird die [pymongo](https://pypi.org/project/pymongo/) Bibliothek genutzt.

Nachdem alle Daten in der Datenbank sind, folgt die Auswertung der Daten. Um aus den Daten eine visuelle Darstellung in Form einer Heatmap aufbauen zu können, kommt die [folium](https://github.com/python-visualization/folium) Bibliothek zum Einsatz.

Es folgt die Installation der genannten Bibliotheken über `pip install`:

In [ ]:
pip install kafka-python pymongo folium

### Importieren benötigter Module

Damit die verschiedenen Funktionen und Datentypen der Bibliotheken genutzt werden können, müssen die entsprechenden Module importiert werden. Im folgenden Abschnitt werden alle notwendigen Module, die im Laufe des Projektes genutzt werden importiert.

Zusätzlich hat sich während der Entwicklung gezeigt, dass je nach Betriebssystem Unterschiede bezüglich der Paralellisierung von Prozessen entstehen. Aus diesem Grund wird hier eine Variable `windows` deklariert, mit der während der Ausführung bestimmter Funktionen zwischen Windows und anderen Betriebssystemen unterschieden werden kann.

Bei der `DEBUG` Variable handelt es sich um eine Variable, welche beim Wert `TRUE` dafür sorgt, dass nur ein Bruchteil der Daten aus den CSV-Dateien importiert wird. Der Import aller Daten (`DEBUG=FALSE`) kann mehrere Stunden in Anspruch nehmen, da es über 9 Millionen Zeilen sind (Stand 12.11.2020). `Mit DEBUG=TRUE` Variable dauert der Import der Daten nur wenige Minuten.

Zuletzt wird hierbei die Breite des Jupyter Notebooks auf 100% gestellt, um die Lesbarkeit zu verbessern.

In [ ]:
from kafka import KafkaProducer, KafkaConsumer
from pymongo import MongoClient
import datetime as dt
import requests
import os
from pathlib import Path
from multiprocessing import Process
from time import sleep
from data.schemas import schemas
import platform
import folium
windows = True if 'Windows' in platform.system() else False 
DEBUG = True

from IPython.core.display import display, HTML
display(HTML(
    "<style>.container { width:100% !important; }</style>"
))

## Verarbeitung der Daten

Um mit der Verarbeitung der Daten zu beginnen, müssen die Daten zunächst aufbereitet und in die MongoDB eingefügt werden. Damit mit Hilfe der MongoDB verschiedene Analysen durchgeführt werden können, sind verschiedene Schritte notwendig, welche in diesem Kapitel nacheinander erläutert und durchgeführt werden.

### Herunterladen der Datensätze

Da die Daten nur verarbeitet werden können, während sie diesem Notebook zur Verfügung stehen, müssen die Daten im ersten Schritt heruntergeladen werden. Hierzu werden die Daten von der Datenquelle mithilfe des nachfolgenden Python-Scripts heruntergeladen. Resultat sind drei `.csv`-Dateien, welche die Unfalldaten zeilenweise enthalten.
Die Dateien werden im Ordner `data` gespeichert, damit sie nur einmalig heruntergeladen werden müssen. Der Ordner befindet sich im gleichen Verzeichnis wie dieses Jupyter Notebook.

In [ ]:
if not os.path.exists('data/'):
    os.mkdir('data')

for file_name, download_url in [
    ('crashes.csv', 'https://data.cityofnewyork.us/api/'
     'views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'),
    ('vehicles.csv', 'https://data.cityofnewyork.us/api/'
     'views/bm4k-52h4/rows.csv?accessType=DOWNLOAD'),
    ('persons.csv', 'https://data.cityofnewyork.us/api/'
     'views/f55k-p6yu/rows.csv?accessType=DOWNLOAD'),
]:
    if not os.path.isfile(fp:= (Path('data') / file_name)):
        with open(fp, 'wb') as crash_file:
            crash_file.write(requests.get(download_url).content)

### Produzieren der CSV-Daten in Kafka

Nachdem die Daten heruntergeladen wurden, beginnt die eigentliche Verarbeitung.
Im folgenden Codeabschnitt wird hierfür die Funktion `send_to_kafka` genutzt, die in das Modul `pipeline_tools.py` ausgelagert wurde. 
Die Funktion erstellt zunächst eine Instanz des `KafkaProducer`, entsprechend des angegebenen Topics.
Um die Daten zu produzieren muss zeilenweise über die entsprechende `.csv` Datei iteriert werden, wobei jedoch ein Laden der gesamten Datei in den RAM vermieden wird, da es sich um einen Generator handelt.
Nachdem eine Zeile ausgelesen wurde, wird diese in das entsprechende `topic` gesandt und muss von einem `KafkaConsumer` wieder ausgelesen werden, um die Daten nutzen zu können.

Für jedes der 3 Topics wird ein eigener Producer Process definiert.
Die einzelnen Prozesse können auf UNIX-basierten Systemen parallel ausgeführt werden, wodurch die Ausführung durch Nutzung von mehr logischen Prozessoren beschleunigt wird. 
Hierbei werden gezielt Prozesse gestartet und keine Threads, da Multithreading in Python bei Ausführung nicht-atomarer Funktionen mit Race Conditions verbunden ist.

In [ ]:
from pipeline_tools import send_to_kafka
                
producers = {
    'crashes': Process(
        target=send_to_kafka, args=[
            'crashes', 15000 if DEBUG else None
    ]),
    'vehicles': Process(
        target=send_to_kafka, args=[
            'vehicles', 5000 if DEBUG else None
    ]),
    'persons': Process(
        target=send_to_kafka, args=[
            'persons', 5000 if DEBUG else None
    ]),
}

### Konsumieren der produzierten Daten und Import in die Datenbank

Im nächsten Schritt wird die Funktion aufgerufen, mit der die Daten aus Kafka ausgelesen und in die MongoDB eingefügt werden. Diese Funktion befindet sich ebenfalls im Modul `pipeline_tools.py`.

In der Funktion wird zunächst eine Verbindung zur MongoDB aufgebaut.
Sobald die Verbindung zur Datenbank bereit ist, wird ein `KafkaConsumer` erstellt, der das übergebene `topic` abonniert. 
Durch das Abonnieren eines Topics erhält der Consumer zeilenweise die zuvor in das Topic gelesenen Daten. 
Jede Zeile wird nach dem Einlesen zunächst in eine Liste konvertiert, indem ein CSV-Reader darauf angewandt wird. 
Da es sich um Daten im CSV-Format handelt, die Strings mit KOmmas enthalten, würde ein Aufruf von `split(',')` fehlerhafte Ergebnisse liefern.

Durch das Trennen in die einzelnen Bestandteile, sind die Daten nun bereit als Key-Value Paare in das entsprechende Dokument innerhalb der MongoDB persistiert zu werden.
Dabei wird das mitgegebene Schema verwendet um die Datentypen bei Bedarf zu konvertieren.

In [ ]:
from pipeline_tools import process_topic

consumers = {
    'vehicles': Process(
        target=process_topic, args=(
        'vehicles', schemas['vehicles']
    )),
    'persons': Process(
        target=process_topic, args=(
        'persons', schemas['persons']
    )),
    'crashes': Process(
        target=process_topic, args=(
        'crashes', schemas['crashes']
    )),
}

### Starten der Prozesse

Nachdem die Funktionen, die für das Speichern der CSV-Daten in die Datenbank zuständig sind, nun fertig definiert sind, werden diese hier aufgerufen. Da die Daten zu Personen und Fahrzeugen (Topic `nyc_persons` und `nyc_vehicles`) im Anschluss an das Persistieren in der Datenbank mit den Unfällen (`crashes`) verbunden werden müssen, werden sie vollständig verarbeitet, bevor die Daten zu den Unfällen in die Datenbank geschrieben werden. 
Dies wurde für das Windows Betriebssystem und andere Betriebssysteme unterschiedlich umgesetzt:

Bei der Ausführung der Funktionen wird zwischen Microsoft Windows und anderen Betriebssystemen unterschieden:
- Windows:<br>
    Da es nicht möglich ist mehrere Child Processes im Jupyter Notebook zu starten, müssen die Funktionsaufrufe nacheinander und blockierend erfolgen. Durch die Synchronität können direkt alle Funktionen aufgerufen werden.
- UNIX:<br>
    Hier werden die Producer für die Topics `nyc_crashes`, `nyc_vehicles`, `nyc_persons` parallel gestartet. Außerdem werden die Consumer der Topics `nyc_vehicles` und `nyc_persons` kurz darauf gestartet. Sobald die beiden Consumer-Prozesse fertig sind, was daran erkenntlich wird, wenn sie nicht mehr in die Datenbank schreiben, können die nachfolgenden Prozesse manuell gestartet werden.

In [ ]:
from pipeline_tools import aggregate_data

if windows:
    limit = send_to_kafka('vehicles', 5000 if DEBUG else None)
    process_topic('vehicles', schemas['vehicles'], limit)
    limit = send_to_kafka('persons', 5000 if DEBUG else None)
    process_topic('persons', schemas['persons'], limit)
    limit = send_to_kafka('crashes', 15000 if DEBUG else None)
    process_topic('crashes', schemas['crashes'], limit)
    
    aggregate_data()
else:
    for topic in 'persons vehicles crashes'.split(' '):
        producers[topic].start()
    sleep(2)
    for topic in 'persons vehicles'.split(' '):
        consumers[topic].start()

### Starten des Join-Prozesses (UNIX)
Im letzten Schritt muss der Prozess gestartet werden, der das Verbinden der beiden Tabellen `vehicles` und `persons` in die Daten des Topics `nyc_crashes` durchführt. Dieser Schritt wurde auf Windows bereits im vorherigen Codeabschnitt erledigt, da dieser auf Windows nicht paralellisiert wird. Das heißt, dass die Vollständigkeit der Daten bereits durch Synchronität gewährleistet wird.

Für UNIX-Systeme ist wichtig, dass die parallelisierten Prozesse zum Produzieren aller Daten und zum Einlesen der `nyc_vehicles` und `nyc_persons` Topics beendet sind. Die Ausführung des vorherigen Codeblocks sollte also abgeschlossen sein. Damit die folgende Funktion bei einem `Kernel` → `Restart & Run All` nicht zu vor Beendigung der geforkten Prozesse ausgeführt wird, wurden 2 Breakpoints eingefügt.

In [ ]:
if not windows:
    assert False, 'non-Windows breakpoint'

In [ ]:
# Ausführung erst nach Beendigung der drei Producer
if not windows:
    consumers['crashes'].start()

In [ ]:
if not windows:
    assert False, 'non-Windows breakpoint'

In [ ]:
# Ausführung erst nach Beendigung des `nyc_crashes` Consumers
if not windows:
    aggregate_data()

# Datenanalyse

Da an dieser Stelle alle Daten vollständig in die Datenbank importiert und anschließend so aufbereitet wurden, dass sie für eine visuelle Analyse verwendet werden können, beginnt an dieser Stelle die Datenanalyse. Wie im Kapitel `Analyseziele` bereits erwähnt, werden zur visuellen Darstellung Diagramme und eine Heatmap genutzt, welche vor der Erstellung nochmal detailliert erläutert werden.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 25, 13

In [ ]:
from pymongo import MongoClient
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["nyc_crashes"]
    crashes = database.crashes
    crashes_by_year = database.crashes_by_year

In [ ]:
def yearly(Year):
    data = crashes_by_year.find_one({"_id": Year})['by_month']
    
    x_values = data.keys()
    y_values_injured = []
    y_values_killed = []
    for month in data:
        injured = 0
        killed = 0
        injured = data[month]['pedestrians']['injured'] + data[month]['cyclists']['injured'] + data[month]['motorists']['injured']
        killed = data[month]['pedestrians']['killed'] + data[month]['cyclists']['killed'] + data[month]['motorists']['killed']
        y_values_injured += [injured]
        y_values_killed += [killed]
    
    show_graph_injury(x_values, y_values_injured, y_values_killed)
    return

In [ ]:
def yearly_sum(Year):
    data = crashes_by_year.find_one({"_id": Year})['by_month']
    
    x_values = data.keys()
    y_values_pedestrians = []
    y_values_cyclists = []
    y_values_motorists = []
    for month in data:
        pedestrians = 0
        cyclists = 0
        motorists = 0
        pedestrians = data[month]['pedestrians']['sum']
        cyclists = data[month]['cyclists']['sum']
        motorists = data[month]['motorists']['sum']
        y_values_pedestrians += [pedestrians]
        y_values_cyclists += [cyclists]
        y_values_motorists += [motorists]
    
    show_graph_type(x_values, y_values_pedestrians, y_values_cyclists, y_values_motorists)
    return

In [ ]:
def show_graph_injury(x_values, y_values_injured, y_values_killed):
    plt.figure(figsize=(20,10))

    plt.bar(x_values, y_values_killed, color=(0.62,0.07,0.04))
    plt.bar(x_values, y_values_injured, bottom=y_values_killed, color=(0.80,0.52,0.00))
    plt.show()

In [ ]:
def show_graph_type(x_values, y_values_pedestrians, y_values_cyclists, y_values_motorists):
    plt.figure(figsize=(20,10))
    
    p1 = plt.bar(x_values, y_values_motorists, color=(0.80,0.50,0.20))
    p2 = plt.bar(x_values, y_values_cyclists, bottom=y_values_motorists, color=(0.75,0.75,0.75))
    bottom_gold = [a+b for a,b in zip(y_values_cyclists, y_values_motorists)]
    p3 = plt.bar(x_values, y_values_pedestrians, bottom=bottom_gold, color=(1.00,0.84,0.00))
    
    plt.ylabel('Anzahl Unfaelle')
    plt.title('Unfaelle pro Monat nach Art')
    plt.legend((p3[0], p2[0], p1[0]), ('Pedestrians', 'Cyclists', 'Motorists'))
    plt.show()

In [ ]:
interact(yearly, Year=[data['_id'] for data in crashes_by_year.find().sort('_id', -1)]);

In [ ]:
interact(yearly_sum, Year=[data['_id'] for data in crashes_by_year.find().sort('_id', -1)]);

## Interaktive Unfallkarte

Mit der Unfallkarte ist es möglich, die Daten nach verschiedenen Unfallkriterien zu untersuchen. Beispiel hierfür sind zum Beispiel die verletzten Fahrradfahrer oder die getöteten Fußgänger.

Die Unfallkarte wird mit Hile der [folium](https://github.com/python-visualization/folium) Bibliothek erstellt und benötigt zur Erstellung lediglich die benötigten Daten und ein wenig Konfiguration.

<h4 style="color:red">Könnte dann nochmal mehr geschrieben werden, wenn das Kartenupdate mit filtern da ist</h4>

In [ ]:
from analysis_tools import crash_map 

In [ ]:
crash_map

# Ergebnis